In [9]:
import pandas as pd
import numpy as np
import os
import time
from collections import Counter
import random

In [2]:
questions = pd.read_csv(os.path.join('EdNet-Contents', 'contents', 'questions.csv'))
question_set=questions["question_id"]
len(question_set)

13169

In [3]:
questions=questions[["question_id","correct_answer"]]
questions

,question_id,correct_answer
0,q1,b
1,q2,a
2,q3,b
3,q4,b
4,q5,c
...,...,...
13164,q18139,b
13165,q18140,a
13166,q18141,a
13167,q18142,a


In [6]:
kt1_files = os.listdir('KT1_joined')
kt1_paths = [os.path.join('KT1_joined', file) for file in kt1_files]

In [55]:
# a=[]
# for i, path in enumerate(kt1_paths):
#     if(i%100000==0):
#         print(i)
#     user_df=pd.read_csv(path)
#     l=len(set(user_df["question_id"]))
#     if(l==100):
#         user_df.to_csv("KT1_selected")
#     a.append(l)

In [56]:
indices = [i for i, x in enumerate(a) if x ==100]
for i in indices:
    user_df=pd.read_csv(kt1_paths[i])
    l=len(set(user_df["question_id"]))
    if(l!=100):
        print("haha")
    if(random.uniform(0,1)<0.2):
        user_df.to_csv(os.path.join('KT1_small',kt1_files[i]),index=False)

In [10]:
li = []
for i, path in enumerate(kt1_paths):
#     if(i%5000==0):
#         print(i)
    user_df = pd.read_csv(path)
    username = kt1_files[i].replace(".csv", "")
    user_df['username'] = username
    li.append(user_df)
# Concatenanting user dfs
kt1 = pd.concat(li, axis = 0 , ignore_index = True)
kt1.to_csv('KT1_joined_large/KT1_part{}.csv'.format(0))

In [66]:
# for i, path in enumerate(kt1_paths):
#     user_df=pd.read_csv(kt1_paths[i])
#     user_df2=user_df.join(questions.set_index("question_id"),on="question_id")
#     score=[1 if b else 0 for b in list(user_df2["user_answer"]==user_df2["correct_answer"])]
#     user_df2["score"]=score
#     user_df2.to_csv(os.path.join('KT1_joined',kt1_files[i]),index=False)

In [64]:
student_set=[kt1_files[i].replace(".csv", "") for i, path in enumerate(kt1_paths)]
SS=len(student_set)
user_df

,Unnamed: 0,timestamp,solving_id,question_id,user_answer,elapsed_time,correct_answer,score,username
0,0,1515924227146,1,q8098,b,12000,b,1,u100562
1,1,1515924253425,2,q8074,b,20000,c,0,u100562
2,2,1515924279087,3,q176,d,23000,d,1,u100562
3,3,1515924297559,4,q1279,b,16000,c,0,u100562
4,4,1515924375098,5,q2067,c,24666,b,0,u100562
...,...,...,...,...,...,...,...,...,...
26483,26483,1531558598616,72,q4297,c,30000,c,1,u99799
26484,26484,1531558774364,73,q6345,a,8000,a,1,u99799
26485,26485,1531558871916,74,q5067,b,21000,b,1,u99799
26486,26486,1531559143446,75,q10159,b,24000,d,0,u99799


In [14]:
user_df=pd.read_csv('KT1_joined_large/KT1_part0.csv')
question_set=set(user_df["question_id"])
len(question_set)

5389

In [18]:
#split questions into train and test sets
def split_questions(path):
    user_df = pd.read_csv(path)
    qq=list(set(user_df["question_id"]))
    n=int(len(qq)/2)
    return qq[0:n],qq[n:len(qq)]

In [23]:
difficulty_ranking={}
start = time.time()
ranking=[]
for q in question_set:
    user_df2=user_df[user_df["question_id"]==q]
    n=len(user_df2)
    #First based on scores
    score=user_df2["score"].sum()/n
    #Then tie breaks on time taken
    timetaken=user_df2["elapsed_time"].sum()/n
    ranking.append((q,score,timetaken))
r=sorted(ranking,key=lambda x:(x[1],-x[2]))
difficulty_ranking=[x[0] for x in r]
end = time.time()
print("Reference ranking inferred in "+str(end - start)+' s')

Reference ranking inferred in 13.347683668136597 s


In [50]:
len(difficulty_ranking)

5389

In [42]:
reference_rankings={}
data={}
for i, path in enumerate(kt1_paths):
    username = kt1_files[i].replace(".csv", "")
    reference_rankings[username]=[]
    data[username]=[]
    train,test=split_questions(path)
    ranking_train=[q for q in difficulty_ranking if q in train]
    ranking_test=[q for q in difficulty_ranking if q in test]
    reference_rankings[username].append(ranking_train)
    reference_rankings[username].append(ranking_test)
    data[username].append(train)
    data[username].append(test)

In [49]:
for i, path in enumerate(kt1_paths):
    username = kt1_files[i].replace(".csv", "")
    if(set(reference_rankings[username][0])!=set(data[username][0])):
        print("haha")
    if(set(reference_rankings[username][1])!=set(data[username][1])):
        print("hehe")

In [65]:
#Check whether one question is more difficult than the other
def rank(q1,q2,ranking):
    if(q1 not in ranking or q2 not in ranking):
        return False
    #return true if q1 is more difficult than q2
    return ranking.index(q1)<ranking.index(q2)

#win score over question pairs q1,q2 given a ranking 
def gamma(q1,q2,ranking):
    if(rank(q1,q2,ranking)):
        return 1
    if(rank(q2,q1,ranking)):
        return -1
    return 0

#all question pairs in L such that qj>qk
def Zk(L,ranking,k):
    answer=[]
    for q in L:
        if(rank(q,k,ranking)):
            answer.append((q,k))
    return answer

#Indicator function
def IA(ranking1,ranking2,q1,q2):
    return 1 if (rank(q1,q2,ranking1) and rank(q1,q2,ranking2)) else 0

#normalized agreement score between two rankings
def Ak(L,ranking1,ranking2,k):
    Z=Zk(L,ranking2,k)
    if(len(Z)==0):
        return 0
    answer=0
    for (q1,q2) in Z:
        answer+=IA(ranking1,ranking2,q1,q2)
    return answer/(L.index(k))
#     return answer/len(Z)

# AP score of ranking2 over L given reference ranking ranking1
def SAP(L,ranking1,ranking2):
    answer=0
    n=len(L)
    for i in range(1,n):
        answer+=Ak(L,ranking1,ranking2,L[i])
    return answer/(n-1)

# def SAP(L,ranking1,ranking2):
#     l=len(L)
#     answer=0
#     for i in range(1,l):
#         s=0
#         for j in range(0,i):
#             for k in range(j,i):
#                 if(k!=j):
#                     s+=IA(ranking1,ranking2,L[j],L[k])
#         s=s/i
#         answer=answer+s
#     return answer/(l-1)

In [57]:
def EduRank(username,ranking,data):
    s=student_set.index(username)
    ranking_train=ranking[username][0]
    ranking_test=ranking[username][1]
    questions_train=data[username][0]
    questions_test=data[username][1]
    t=len(ranking_train)
    M=[0 for i in range(SS)]
    for i in range(SS):
        if(i!=s):
            M[i]=SAP(ranking_train,ranking_train,ranking[student_set[i]][0])
    r=[[] for i in range(t)]
    for i in range(t):
        for j in range(t):
            if(i!=j):
                rv=0
                for k in range(SS):
                    if(k!=s):
                        rv+=M[k]*gamma(questions_test[i],questions_test[j],ranking[student_set[k]][0])
                rv=np.sign(rv)     
                r[i].append(rv)
    dict=[]
    for i in range(t):
        dict.append((questions_test[i],sum(r[i])))
    dict=sorted(dict, key=lambda x:-x[1])
    result=[x[0] for x in dict]
    sap=SAP(ranking_test,ranking_test,result)
    return sap

In [28]:
def EduRank_coldstart(s):
    start = time.time()
    train=data[sset[s]][0]
    test=data[sset[s]][1]
    t=len(train)
    A=[[Ak(train,reference_rankings[sset[s]],reference_rankings[sset[i]],train[k])
        for k in range(t)]for i in range(SS) if i!=s]
    M=[np.mean(A[i]) for i in range(SS-1)]
    r=[[] for i in range(t)]
    for i in range(t):
        for j in range(t):
            rv=0
            s12=[s for s in sset if qset[i] in difficulty_rankings[s] and qset[j] in difficulty_rankings[s]]
            priori=priors_mean[qset[i]]
            priorj=priors_mean[qset[j]]
            alpha=len(s12)/SS
            for k in range(SS):
                if(k!=s):
                    rv+=M[k-1]*gamma(test[i],test[j],difficulty_rankings[sset[k]])
            rv=alpha*np.sign(rv)
            rv+=(1-alpha)*np.sign(priori-priorj)        
            r[i].append(rv)
    dict=[]
    for i in range(len(test)-1):
        dict.append((test[i],sum(r[i])))
    dict=sorted(dict, key=lambda x:-x[1])
    result=[x[0] for x in dict]
    result=SAP(test,difficulty_rankings[username],result)
    end = time.time()
    return result


In [66]:
sap=0
for s in student_set:
    e=EduRank(s,reference_rankings,data)
    sap+=e
sap/=SS
print(sap)

0.5096639662485899


In [62]:
reference_rankings2={}
data2={}
for i, path in enumerate(kt1_paths):
    username = kt1_files[i].replace(".csv", "")
    reference_rankings2[username]=[]
    data2[username]=[]
    train,test=split_questions(path)
    l=len(train)
    train=train[:int(0.1*l)]
    test=test[:int(0.1*l)]
    ranking_train=[q for q in difficulty_ranking if q in train]
    ranking_test=[q for q in difficulty_ranking if q in test]
    reference_rankings2[username].append(ranking_train)
    reference_rankings2[username].append(ranking_test)
    data2[username].append(train)
    data2[username].append(test)

In [63]:
sap=0
for s in student_set:
    sap+=EduRank(s,reference_rankings2,data2)
sap/=SS
print(sap)

0.5023535564853553
